# Roleplay actions
The purpose of this notebook is to seed the model with actions to further stimulate it to react to roleplay actions such as "meet Lorem". The actions are mixed into the model and aren't part of some kind of post processing so more is left to chance. Cause if it were guaranteed, where's the fun in that!

In [1]:
import os
import numpy as np
import pandas as pd
import scipy
import json
import sys
sys.path.append('..')
from config import Config

In [2]:
output_path = os.path.join("..", "rp_data.txt")
d = {
    'Ry': 'remyapt',
    'Lo': 'loremapt',
    'Ip': 'loremapt',
    'Br': 'bare',
    'Em': 'emeraroom',
    'Ka': ['park3', 'park2', 'park1'],
    'Rz': ['park3', 'park2', 'park1'],
    'Kv': ['park3', 'park2', 'park1'],
    'Zh': 'bare',
    'Mv': ['park3', 'park2', 'park1'],
    'An': 'corridor',
    'Ad': 'adineapt',
    'Sb': 'office'
}
{ k: { 'scenes': v } for k, v in d.items() }

{'Ry': {'scenes': 'remyapt'},
 'Lo': {'scenes': 'loremapt'},
 'Ip': {'scenes': 'loremapt'},
 'Br': {'scenes': 'bare'},
 'Em': {'scenes': 'emeraroom'},
 'Ka': {'scenes': ['park3', 'park2', 'park1']},
 'Rz': {'scenes': ['park3', 'park2', 'park1']},
 'Kv': {'scenes': ['park3', 'park2', 'park1']},
 'Zh': {'scenes': 'bare'},
 'Mv': {'scenes': ['park3', 'park2', 'park1']},
 'An': {'scenes': 'corridor'},
 'Ad': {'scenes': 'adineapt'},
 'Sb': {'scenes': 'office'}}

In [3]:
interactable_characters = Config.interactable_characters
interactable_characters_inv = { v: k for k, v in interactable_characters.items() }

class ActionReaction():
    def __init__(self, actions, reactions, characters, generate_lower_case = True, about_character = None):
        self.actions = actions
        self.reactions = reactions
        self.characters = characters
        self.generate_lower_case = generate_lower_case
        self.about_character = about_character
        
    def to_commands(self):
        command_lines = []
        for action in self.actions:
            for reaction in self.reactions:
                for character_code, character_data in self.characters.items():
                    scenes = ['%lastscene']
                    if 'scenes' in character_data:
                        scenes = character_data['scenes']
                        if type(scenes) is not list:
                            scenes = [scenes]
                    for scene in scenes:      
                        cmd_json = {
                        }
                        if self.about_character is not None:
                            cmd_json['about_character'] = self.about_character.replace("%character_code", character_code)
                        character_name = character_code
                        if character_code in interactable_characters:
                            character_name = interactable_characters[character_code]
                        cmd_json['cmd'] = f'<p><msg>c "{action.replace("%character", character_name)}"<d><scn>{scene}<msg>{character_code} "{reaction}"'
                        command_lines.append(json.dumps(cmd_json))
                        if self.generate_lower_case:
                            lower1 = f'<p><msg>c "{action.replace("%character", character_name).lower()}"<d><scn>{scene}<msg>{character_code} "{reaction}"'
                            cmd_json['cmd'] = lower1
                            lower2 = f'<p><msg>c "{action.lower().replace("%character", character_name).lower()}"<d><scn>{scene}<msg>{character_code} "{reaction}"'
                            command_lines.append(json.dumps(cmd_json))
                            if lower1 != lower2:
                                cmd_json['cmd'] = lower2
                                command_lines.append(json.dumps(cmd_json))
        return command_lines

# Character actions
These actions relate to characters, each action will be paired to a character response.

In [5]:
actions = [
    ActionReaction([
        "Meet with %character",
        "Visit %character",
        "Go to %character",
    ], [
        "Hey [player_name]!",
        "Hey [player_name]! How are you?",
        "Oh, [player_name], I didn't expect you to arrive so early.",
        "Oh, [player_name], I wasn't expecting visitors.",
    ], 
    {
        'Ry': {'scenes': 'remyapt'},
        'Lo': {'scenes': 'loremapt'},
        'Ip': {'scenes': 'loremapt'},
        'Br': {'scenes': 'bare'},
        'Em': {'scenes': 'emeraroom'},
        'Ka': {'scenes': ['park3', 'park2', 'park1']},
        'Rz': {'scenes': ['park3', 'park2', 'park1']},
        'Kv': {'scenes': ['park3', 'park2', 'park1']},
        'Zh': {'scenes': 'bare'},
        'Mv': {'scenes': ['park3', 'park2', 'park1']},
        'An': {'scenes': 'corridor'},
        'Ad': {'scenes': 'adineapt'},
        'Sb': {'scenes': 'office'}
    }, about_character="%character_code"),
    ActionReaction([
        "Leave",
        "Go to park",
    ], [
        "Hey [player_name]!",
        "Hey [player_name]! How are you?",
        "Oh, [player_name], I didn't expect you to arrive so early.",
        "Oh, [player_name], I wasn't expecting visitors.",
    ], 
    {
        'Ry': {'scenes': 'remyapt'},
        'Lo': {'scenes': 'loremapt'},
        'Ip': {'scenes': 'loremapt'},
        'Br': {'scenes': 'bare'},
        'Em': {'scenes': 'emeraroom'},
        'Ka': {'scenes': ['park3', 'park2', 'park1']},
        'Rz': {'scenes': ['park3', 'park2', 'park1']},
        'Kv': {'scenes': ['park3', 'park2', 'park1']},
        'Zh': {'scenes': 'bare'},
        'Mv': {'scenes': ['park3', 'park2', 'park1']},
        'An': {'scenes': 'corridor'},
        'Ad': {'scenes': 'adineapt'},
        'Sb': {'scenes': 'office'}
    }),
    ActionReaction([
        "Fight",
        "Attack"
    ], [
        "%lastcharacter barely avoids my attack and fell, but managed to get up and quickly punch me in the face, a soaring pain quickly came over my face",
        "%lastcharacter dodges my attack and comes rushing towards me",
        "I didn't hesitate and kicked %lastcharacter right in the stomach"
    ], 
    {
        'm': {'scenes': '%lastscene'},
    }),
    ActionReaction([
        "Kick",
        "Punch",
        "Bite",
    ], [
        "Wtf you think you doing?",
        "Think you can hurt me?",
        "Stop it now or I'll fuck you up",
        "Holy shit!"
    ], 
    {
        '%lastcharactercode': {
            'scenes': '%lastscene'
        }
    }),
    ActionReaction([
        "Hey %character! Over here!",
        "Hello %character",
        "%character? Is that you?",
    ], [
        "Hey [player_name]!",
        "Hey [player_name]! How are you?",
        "[player_name]? What are you doing here?",
    ], 
    {
        k: {'scenes': '%lastscene'} for k, v in interactable_characters.items()
    }, about_character="%character_code")
]

character_metadata = {
    'Lo': {
        'gender': 'm'
    },
    'Ad': {
        'gender': 'f'
    },
    'An': {
        'gender': 'f'
    },
    'Br': {
        'gender': 'm'
    },
    'Ry': {
        'gender': 'm'
    },
    'Ka': {
        'gender': 'm'
    },
    'Sb': {
        'gender': 'm'
    },
    'Kv': {
        'gender': 'm'
    },
    'Ip': {
        'gender': 'm'
    },
    'Em': {
        'gender': 'f'
    },
    'Zh': {
        'gender': 'm'
    }
}

pronouns_mapping = {
    'm': ['he', 'him', 'He'],
    'f': ['she', 'her', 'She']
}
    
for k in character_metadata:
    character = character_metadata[k]
    pronouns = pronouns_mapping[character['gender']]
    name = Config.interactable_characters[k]
    actions.append(ActionReaction([
        f"What do you think of {name}?"
    ], [
        f"I think {pronouns[0]}'s cute.",
        f"I dont like {pronouns[1]} as well as before.",
        f"I like {pronouns[1]}.",
        f"{pronouns[2]} is a bit odd.",
        f"{pronouns[2]} is a bit interesting.",
        f"I think {pronouns[0]}'s funny.",
        f"I think {pronouns[0]}'s ugly.",
        f"I think {pronouns[0]}'s good looking.",
        f"I think {pronouns[0]} is funny.",
        f"I think {pronouns[0]} is ugly.",
        f"I think {pronouns[0]} is good looking.",
    ], 
    {
        '%lastcharactercode': {
            'scenes': '%lastscene'
        }
    }, generate_lower_case = False, about_character=k))
    actions.append(ActionReaction([
        f"I was with {name} today"
    ], [
        "Very nice"
    ], 
    {
        '%lastcharactercode': {
            'scenes': '%lastscene'
        }
    }, generate_lower_case = False, about_character=k))

with open(output_path, 'w') as f:
    command_lines = []
    for action in actions:
        for cmd in action.to_commands():
            f.write(cmd + "\n")